<a href="https://colab.research.google.com/github/wilberquito/AMLProject/blob/main/AMLProject.wil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Project: Image Classification
## Advanced Machine Learning
## Cifar 10

> Wilber E. Bermeo Quito 
>
> Judit Quintana Massana
>
> April 2023

In [4]:
import zipfile
from pathlib import Path
import torch
import matplotlib.pyplot as plt

In [5]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [6]:
if IN_COLAB:
    ! pip install torchvision
    ! pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    !rm -rf data
    data_path = Path('/content/drive/MyDrive/AML/dataset_CIFAR10.zip')
    with zipfile.ZipFile(data_path,"r") as zip_ref:
        zip_ref.extractall("data")

    !rm -rf modular
    data_path = Path('/content/drive/MyDrive/AML/modular.zip')
    with zipfile.ZipFile(data_path,"r") as zip_ref:
        zip_ref.extractall(".")

Mounted at /content/drive


## AML Resnet50 v0

In [ ]:
import modular.models as models

amlresnet50_v0 = models.AMLResnet50_V0(out_dim=10)

from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: 
          continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(amlresnet50_v0)

+-------------+------------+
|   Modules   | Parameters |
+-------------+------------+
| fc.0.weight |   262144   |
|  fc.0.bias  |    128     |
| fc.2.weight |    1280    |
|  fc.2.bias  |     10     |
+-------------+------------+
Total Trainable Params: 263562


263562

In [ ]:
import torch
import modular.datasets as datasets
import modular.models as models 
from pathlib import Path
from modular.engine import train
import torchvision.transforms as transforms
from modular.utils import set_seeds

set_seeds(seed=42)

# Model
amlresnet50_v0 = models.AMLResnet50_V0(out_dim=10)

train_transforms, validate_transforms = amlresnet50_v0.transforms, amlresnet50_v0.transforms

augmentation = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ColorJitter(),
    transforms.RandomGrayscale(),
    transforms.RandomHorizontalFlip(),
])

train_transforms = transforms.Compose([augmentation, train_transforms])

# Optimizer
optimizer = torch.optim.Adam(params=amlresnet50_v0.fc.parameters())

# Criterion
criterion = torch.nn.CrossEntropyLoss()

# Number of epochs to train the model
epochs = 20

# Default device to train model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

# Where the model is saved
save_as = Path('trained/amlresnet50_v0.pth')

# Mini batch
batch_size=800

train_dataloader = datasets.get_dataloader(folder_root='data/train',
                                           transformer=train_transforms,
                                           batch_size=batch_size,
                                           suffle=True)
validate_dataloader = datasets.get_dataloader(folder_root='data/validation',
                                           transformer=validate_transforms,
                                           batch_size=batch_size,
                                           suffle=False)

In [ ]:
results = train(model=amlresnet50_v0,
                train_dataloader=train_dataloader,
                test_dataloader=validate_dataloader,
                optimizer=optimizer,
                criterion=criterion,
                epochs=epochs,
                device=device,
                save_as=save_as)

  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
results

## AML Resnet50 v0 LR

In [ ]:
import torch
import modular.datasets as datasets
import modular.models as models 
from pathlib import Path
from modular.engine import train
import torchvision.transforms as transforms
from modular.utils import set_seeds

set_seeds(seed=42)

# Model
amlresnet50_v0 = models.AMLResnet50_V0(out_dim=10)

train_transforms, validate_transforms = amlresnet50_v0.transforms, amlresnet50_v0.transforms

augmentation = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ColorJitter(),
    transforms.RandomGrayscale(),
    transforms.RandomHorizontalFlip(),
])

train_transforms = transforms.Compose([augmentation, train_transforms])

# Define your optimizer
optimizer = torch.optim.Adam(amlresnet50_v0.parameters(), lr=0.005)

# Define your learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Criterion
criterion = torch.nn.CrossEntropyLoss()

# Number of epochs to train the model
epochs = 20

# Default device to train model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

# Where the model is saved
save_as = Path('trained/amlresnet50_v0.step.pth')

# Mini batch
batch_size=800

train_dataloader = datasets.get_dataloader(folder_root='data/train',
                                           transformer=train_transforms,
                                           batch_size=batch_size,
                                           suffle=True)
validate_dataloader = datasets.get_dataloader(folder_root='data/validation',
                                           transformer=validate_transforms,
                                           batch_size=batch_size,
                                           suffle=False)

In [ ]:
results = train(model=amlresnet50_v0,
                train_dataloader=train_dataloader,
                test_dataloader=validate_dataloader,
                optimizer=optimizer,
                criterion=criterion,
                epochs=epochs,
                device=device,
                scheduler=scheduler,
                save_as=save_as)

  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Epoch: 1 | train_loss: 1.4205 | train_acc: 0.4938 | test_loss: 1.1236 | test_acc: 0.6115


0it [00:00, ?it/s]

...

Epoch: 4 | train_loss: 1.0664 | train_acc: 0.6219 | test_loss: 0.9924 | test_acc: 0.6553


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Epoch: 5 | train_loss: 1.0484 | train_acc: 0.6260 | test_loss: 0.9724 | test_acc: 0.6569


0it [00:00, ?it/s]

In [ ]:
from torchvision.models import resnet101, ResNet101_Weights

resnet101(weights=ResNet101_Weights.IMAGENET1K_V2)

## Resnet 101 V0

In [1]:
import torch
import modular.datasets as datasets
import modular.models as models 
from pathlib import Path
from modular.engine import train
import torchvision.transforms as transforms
from modular.utils import set_seeds

set_seeds(seed=42)

# Model
model = models.AMLResnet101_V0(out_dim=10)

train_transforms, validate_transforms = model.transforms, model.transforms

augmentation = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ColorJitter(),
    transforms.RandomGrayscale(),
    transforms.RandomHorizontalFlip(),
])

train_transforms = transforms.Compose([augmentation, train_transforms])

# Optimizer (REDUCE EL LEARNING RATE IN TRANSFER LEARNING)
optimizer = torch.optim.Adam(params=model.fc.parameters(), lr=0.01)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Criterion
criterion = torch.nn.CrossEntropyLoss()

# Number of epochs to train the model
epochs = 5

# Default device to train model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

# Where the model is saved
save_as = Path('trained/amlresnet101.pth')

# Mini batch
batch_size=100

train_dataloader = datasets.get_dataloader(folder_root='data/train',
                                           transformer=train_transforms,
                                           batch_size=batch_size,
                                           suffle=True)
validate_dataloader = datasets.get_dataloader(folder_root='data/validation',
                                           transformer=validate_transforms,
                                           batch_size=batch_size,
                                           suffle=False)

In [12]:
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=validate_dataloader,
                optimizer=optimizer,
                criterion=criterion,
                epochs=epochs,
                device=device,
                save_as=save_as)

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Epoch: 1 | train_loss: 1.4886 | train_acc: 0.4757 | test_loss: 1.0889 | test_acc: 0.6228


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

Epoch: 4 | train_loss: 1.2475 | train_acc: 0.5585 | test_loss: 1.0143 | test_acc: 0.6472


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Epoch: 5 | train_loss: 1.2264 | train_acc: 0.5639 | test_loss: 1.0077 | test_acc: 0.6483


In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
from torchinfo import summary
import modular.models as models 
from pathlib import Path
import torch
from modular.engine import train

path = Path('trained/amlresnet101.pth')
model = models.AMLResnet101_V0(out_dim=10)
model.load_state_dict(torch.load(path))
model


In [3]:
save_as = Path('trained/warm.amlresnet101.pth')
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

model.unfreeze_base()
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=validate_dataloader,
                optimizer=optimizer,
                criterion=criterion,
                epochs=epochs,
                device=device,
                save_as=save_as)

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Epoch: 1 | train_loss: 0.7879 | train_acc: 0.7263 | test_loss: 0.5767 | test_acc: 0.8019


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

Epoch: 4 | train_loss: 0.4456 | train_acc: 0.8445 | test_loss: 0.3906 | test_acc: 0.8680


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Epoch: 5 | train_loss: 0.3971 | train_acc: 0.8632 | test_loss: 0.4198 | test_acc: 0.8579


In [4]:
efficientnet...
resnext...

SyntaxError: ignored